In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from shutil import copyfile
import copy
import time
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models
import numpy as np
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score

In [ ]:
# 1. Defining transformations for training data
transform_1_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize with ImageNet statistics
])

# 2. Defining transformations for test and validation data
transform_1_test = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

# 3. Loading the training dataset with transformations applied
train_model_1_dataset = datasets.Flowers102(root='data', split='train', download=True, transform = transform_1_train)
val_model_1_dataset = datasets.Flowers102(root='data', split='val', download=True, transform = transform_1_test)
test_model_1_dataset = datasets.Flowers102(root='data', split='test', download=True, transform = transform_1_test)

In [ ]:
# 1. Creating DataLoader for training , validation, test datasets
train_model_1_dataloader = DataLoader(train_model_1_dataset, batch_size=32, shuffle=True)
val_model_1_dataloader = DataLoader(val_model_1_dataset, batch_size=32, shuffle=False)
test__model_1_dataloader = DataLoader(test_model_1_dataset, batch_size=32, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)  # Layer 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Layer 2

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # Layer 3
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Layer 4

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)  # Layer 5
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Layer 6

        self.flatten = nn.Flatten()  # Flatten the tensor starting from the first dimension (batch dimension)

        # Fully connected layers
        self.fc1 = nn.Linear(in_features=128 * 32 * 32, out_features=512) # Layer 7
        self.fc2 = nn.Linear(in_features=512, out_features=102)  # Layer 8 (output layer, 102 classes)

    def forward(self, x):
        # Forward pass through the network
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))

        x = self.fc2(x) # Output layer
        return x

# Create an instance of the model
model1 = SimpleCNN()
print(model1)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=131072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=102, bias=True)
)


In [ ]:
# Installing tensorboard and torchinfo
!pip install torchinfo
!pip install tensorboard

In [ ]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

model1 = SimpleCNN()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1.to(device) # Move the model to the specified device

optimizer = optim.SGD(model1.parameters(), lr=0.001)  # Stochastic Gradient Descent (SGD) with learning rate of 0.001

criterion = nn.CrossEntropyLoss()

# Initialize the TensorBoard writer to log metrics
writer = SummaryWriter(log_dir='runs/flower_classification')


Function to train the model for one epoch
 Args:
    - model (torch.nn.Module): The model to be trained.
    - train_dataloader (DataLoader): DataLoader for the training set.
    - criterion (torch.nn.Module): Loss function used for training.
    - optimizer (torch.optim.Optimizer): Optimizer used for training.
    - device (torch.device): The device (CPU or GPU) to train the model on.
    - writer (SummaryWriter): TensorBoard writer for logging.
    - epoch (int): The current epoch number.
    
    Returns:
    - epoch_loss (float): The average loss for the epoch.
    - epoch_accuracy (float): The accuracy for the epoch.
    - epoch_f1_score (float): The F1 score for the epoch.

In [ ]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device, writer, epoch):

    model.train()  # Set the model to training mode

    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    all_train_preds = []
    all_train_labels = []

    # Loop through the training data
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero out the gradients

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)

        # Update prediction and label lists for F1 score calculation
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        all_train_preds.extend(predicted.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    # Calculate average loss, accuracy, and F1 score for the epoch
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct_preds / total_preds
    epoch_f1_score = f1_score(all_train_labels, all_train_preds, average='weighted')

    # Log metrics to TensorBoard
    writer.add_scalar('Training Loss', epoch_loss, epoch)
    writer.add_scalar('Training Accuracy', epoch_accuracy, epoch)
    writer.add_scalar('Training F1 Score', epoch_f1_score, epoch)

    print(f'Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Train F1 Score: {epoch_f1_score:.4f}')

    return epoch_loss, epoch_accuracy, epoch_f1_score

In [ ]:
def validate_one_epoch(model, val_dataloader, criterion, device, writer, epoch):
    model.eval()

    val_loss = 0.0
    correct_preds_val = 0
    total_preds_val = 0
    all_val_preds = []
    all_val_labels = []

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds_val += (predicted == labels).sum().item()
            total_preds_val += labels.size(0)

            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())


    val_loss = val_loss / len(val_dataloader.dataset)
    val_accuracy = correct_preds_val / total_preds_val
    val_f1_score = f1_score(all_val_labels, all_val_preds, average='weighted')


    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    writer.add_scalar('Validation F1 Score', val_f1_score, epoch)

    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1_score:.4f}')

    return val_loss, val_accuracy, val_f1_score

In [ ]:
def train_model(model, train_dataloader, val_dataloader, test_dataloader, criterion, optimizer, device, num_epochs, writer):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    train_f1_scores = []
    val_f1_scores = []

    for epoch in range(num_epochs):

        epoch_loss, epoch_accuracy, epoch_f1_score = train_one_epoch(
            model, train_dataloader, criterion, optimizer, device, writer, epoch)


        val_loss, val_accuracy, val_f1_score = validate_one_epoch(
            model, val_dataloader, criterion, device, writer, epoch)


        train_losses.append(epoch_loss)
        val_losses.append(val_loss)
        train_accuracies.append(epoch_accuracy)
        val_accuracies.append(val_accuracy)
        train_f1_scores.append(epoch_f1_score)
        val_f1_scores.append(val_f1_score)


    test_loss, test_accuracy, test_f1_score = test_model(model, test_dataloader, criterion, device)

    return (train_losses, val_losses, train_accuracies, val_accuracies,
            train_f1_scores, val_f1_scores, test_loss, test_accuracy, test_f1_score)



In [ ]:
def test_model(model, test_dataloader, criterion, device):
    model.eval()

    test_loss = 0.0
    correct_preds_test = 0
    total_preds_test = 0
    all_test_preds = []
    all_test_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds_test += (predicted == labels).sum().item()
            total_preds_test += labels.size(0)

            all_test_preds.extend(predicted.cpu().numpy())
            all_test_labels.extend(labels.cpu().numpy())


    test_loss = test_loss / len(test_dataloader.dataset)
    test_accuracy = correct_preds_test / total_preds_test
    test_f1_score = f1_score(all_test_labels, all_test_preds, average='weighted')

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1_score:.4f}')

    return test_loss, test_accuracy, test_f1_score

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, train_f1_scores, val_f1_scores, test_loss, test_accuracy, test_f1_score = train_model(
    model=model1,
    train_dataloader = train_model_1_dataloader,
    val_dataloader = val_model_1_dataloader,
    test_dataloader = test__model_1_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=10,
    writer=writer
)

Epoch 1, Train Loss: 4.6264, Train Accuracy: 0.0069, Train F1 Score: 0.0005
Epoch 1, Validation Loss: 4.6232, Validation Accuracy: 0.0127, Validation F1 Score: 0.0014
Epoch 2, Train Loss: 4.6223, Train Accuracy: 0.0098, Train F1 Score: 0.0015
Epoch 2, Validation Loss: 4.6208, Validation Accuracy: 0.0196, Validation F1 Score: 0.0039
Epoch 3, Train Loss: 4.6186, Train Accuracy: 0.0196, Train F1 Score: 0.0056
Epoch 3, Validation Loss: 4.6185, Validation Accuracy: 0.0294, Validation F1 Score: 0.0068
Epoch 4, Train Loss: 4.6149, Train Accuracy: 0.0333, Train F1 Score: 0.0076
Epoch 4, Validation Loss: 4.6161, Validation Accuracy: 0.0294, Validation F1 Score: 0.0061
Epoch 5, Train Loss: 4.6109, Train Accuracy: 0.0412, Train F1 Score: 0.0117
Epoch 5, Validation Loss: 4.6135, Validation Accuracy: 0.0363, Validation F1 Score: 0.0082
Epoch 6, Train Loss: 4.6071, Train Accuracy: 0.0490, Train F1 Score: 0.0179
Epoch 6, Validation Loss: 4.6108, Validation Accuracy: 0.0402, Validation F1 Score: 0.011

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs/flower_classification

In [ ]:
# Очистка кеша GPU после тренировки модели
torch.cuda.empty_cache()

In [ ]:
transform_train = transforms.Compose([transforms.Resize((256,256)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ])
# Трансформации для теста
transform_test = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),               # Преобразование в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])
# Загрузка тренировочного и тестового датасетов
train_model_2_dataset = datasets.Flowers102(root='data', split='train', download=True, transform= transform_train)
val_model_2_dataset = datasets.Flowers102(root='data', split='val', download=True, transform= transform_test)
test_model_2_dataset = datasets.Flowers102(root='data', split='test', download=True, transform= transform_test)

In [ ]:
train_model_2_dataloader = DataLoader(train_model_2_dataset, batch_size=32, shuffle=True)
val_model_2_dataloader = DataLoader(val_model_2_dataset, batch_size=32, shuffle=False)
test_model_2_dataloader = DataLoader(test_model_2_dataset, batch_size=32,shuffle=False)

In [ ]:
!pip install torchinfo

In [ ]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()

        # Слои свёрточной нейронной сети с Batch Normalization
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        # Полносвязный слой с Dropout
        self.fc1 = nn.Linear(in_features=128 * 32 * 32, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=102)

        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Прямой проход через сеть с Batch Normalization и Dropout
        x = F.relu(self.bn1(self.conv1(x)))  # Применяем BN и ReLU
        x = self.pool1(x)

        x = F.relu(self.bn2(self.conv2(x)))  # Применяем BN и ReLU
        x = self.pool2(x)

        x = F.relu(self.bn3(self.conv3(x)))  # Применяем BN и ReLU
        x = self.pool3(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Создание модели
model2 = ImprovedCNN()

print(model2)

ImprovedCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=131072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=102, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
# Создаём модель
model2 = ImprovedCNN()

# Убираем использование GPU (если доступен)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2.to(device)

# Оптимизатор
optimizer = optim.SGD(model2.parameters(), lr=0.001)  # SGD momentum=0.9, weight_decay=1e-4

# Функция потерь для многоклассовой классификации
criterion = nn.CrossEntropyLoss()

# Логирование с использованием TensorBoard
writer = SummaryWriter(log_dir='runs/flower_classification')  # Создаём директорию для TensorBoard


In [ ]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device, writer, epoch):
    model.train()

    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    all_train_preds = []
    all_train_labels = []

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss

        loss.backward()  # Backward pass
        optimizer.step()  # Optimizer step

        running_loss += loss.item() * inputs.size(0)  # Accumulate loss
        _, predicted = torch.max(outputs, 1)  # Get predicted labels
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        all_train_preds.extend(predicted.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    # Compute epoch metrics
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct_preds / total_preds
    epoch_f1_score = f1_score(all_train_labels, all_train_preds, average='weighted')

    # Log metrics to TensorBoard
    writer.add_scalar('Training Loss', epoch_loss, epoch)
    writer.add_scalar('Training Accuracy', epoch_accuracy, epoch)
    writer.add_scalar('Training F1 Score', epoch_f1_score, epoch)

    print(f'Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, F1 Score: {epoch_f1_score:.4f}')

    return epoch_loss, epoch_accuracy, epoch_f1_score


In [ ]:
def validate_one_epoch(model, val_dataloader, criterion, device, writer, epoch):
    model.eval()  # Switch to evaluation mode (turns off Dropout, BatchNorm)

    val_loss = 0.0
    correct_preds_val = 0
    total_preds_val = 0
    all_val_preds = []
    all_val_labels = []

    with torch.no_grad():  # Disable gradient calculation during validation
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss

            val_loss += loss.item() * inputs.size(0)  # Accumulate loss
            _, predicted = torch.max(outputs, 1)  # Get predicted labels
            correct_preds_val += (predicted == labels).sum().item()
            total_preds_val += labels.size(0)

            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    # Compute epoch metrics
    val_loss = val_loss / len(val_dataloader.dataset)
    val_accuracy = correct_preds_val / total_preds_val
    val_f1_score = f1_score(all_val_labels, all_val_preds, average='weighted')

    # Log metrics to TensorBoard
    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    writer.add_scalar('Validation F1 Score', val_f1_score, epoch)

    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1_score:.4f}')

    return val_loss, val_accuracy, val_f1_score


In [ ]:
def test_model(model, test_dataloader, criterion, device):
    model.eval()  # Переключаем модель в режим оценки

    test_loss = 0.0
    correct_preds_test = 0
    total_preds_test = 0
    all_test_preds = []
    all_test_labels = []

    with torch.no_grad():  # Отключаем вычисление градиентов
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  # Прямой проход
            loss = criterion(outputs, labels)  # Вычисляем потерю

            test_loss += loss.item() * inputs.size(0)  # Накопление потерь
            _, predicted = torch.max(outputs, 1)  # Выбираем класс с максимальной вероятностью
            correct_preds_test += (predicted == labels).sum().item()
            total_preds_test += labels.size(0)

            all_test_preds.extend(predicted.cpu().numpy())
            all_test_labels.extend(labels.cpu().numpy())

    # Вычисляем метрики для теста
    test_loss = test_loss / len(test_dataloader.dataset)
    test_accuracy = correct_preds_test / total_preds_test
    test_f1_score = f1_score(all_test_labels, all_test_preds, average='weighted')

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1_score:.4f}')

    return test_loss, test_accuracy, test_f1_score

In [ ]:
def train_model(model, train_dataloader, val_dataloader, test_dataloader, criterion, optimizer, device, num_epochs, writer):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    train_f1_scores = []
    val_f1_scores = []

    for epoch in range(num_epochs):
        # Обучение модели на одной эпохе
        train_loss, train_accuracy, train_f1_score = train_one_epoch(
            model, train_dataloader, criterion, optimizer, device, writer, epoch
        )

        # Валидация модели на одной эпохе
        val_loss, val_accuracy, val_f1_score = validate_one_epoch(
            model, val_dataloader, criterion, device, writer, epoch
        )

        # Логирование метрик на каждой эпохе
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        train_f1_scores.append(train_f1_score)
        val_f1_scores.append(val_f1_score)

    # Тестирование модели после завершения всех эпох
    test_loss, test_accuracy, test_f1_score = test_model(
        model, test_dataloader, criterion, device
    )

    # Закрытие TensorBoard writer
    writer.close()

    return train_losses, val_losses, train_accuracies, val_accuracies, train_f1_scores, val_f1_scores, test_loss, test_accuracy, test_f1_score

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies, train_f1_scores, val_f1_scores, test_loss, test_accuracy, test_f1_score = train_model(
    model = model2,
    train_dataloader = train_model_2_dataloader,
    val_dataloader = val_model_2_dataloader,
    test_dataloader = test_model_2_dataloader,  # Передаем test_dataloader
    criterion = criterion,
    optimizer = optimizer,
    device=device,
    num_epochs=10,
    writer=writer
)

Epoch 1, Train Loss: 4.6528, Accuracy: 0.0196, F1 Score: 0.0138
Epoch 1, Validation Loss: 4.5233, Accuracy: 0.0441, F1 Score: 0.0184
Epoch 2, Train Loss: 4.3734, Accuracy: 0.0422, F1 Score: 0.0337
Epoch 2, Validation Loss: 4.3145, Accuracy: 0.0657, F1 Score: 0.0342
Epoch 3, Train Loss: 4.0980, Accuracy: 0.0922, F1 Score: 0.0774
Epoch 3, Validation Loss: 4.1322, Accuracy: 0.1049, F1 Score: 0.0720
Epoch 4, Train Loss: 3.7950, Accuracy: 0.1578, F1 Score: 0.1412
Epoch 4, Validation Loss: 3.9589, Accuracy: 0.1422, F1 Score: 0.1025
Epoch 5, Train Loss: 3.4312, Accuracy: 0.2451, F1 Score: 0.2230
Epoch 5, Validation Loss: 3.8461, Accuracy: 0.1520, F1 Score: 0.1072
Epoch 6, Train Loss: 3.1290, Accuracy: 0.3206, F1 Score: 0.2989
Epoch 6, Validation Loss: 3.6778, Accuracy: 0.1902, F1 Score: 0.1434
Epoch 7, Train Loss: 2.8044, Accuracy: 0.4039, F1 Score: 0.3873
Epoch 7, Validation Loss: 3.6000, Accuracy: 0.2078, F1 Score: 0.1621
Epoch 8, Train Loss: 2.4092, Accuracy: 0.5294, F1 Score: 0.5161
Epoch

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs/flower_classification

In [ ]:
    # Очистка кеша GPU после тренировки модели
torch.cuda.empty_cache()

In [ ]:
# Определение преобразований
transform_train = transforms.Compose([
        transforms.Resize(256),
    transforms.CenterCrop(224),
  # Crop the center 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    # Normalize with ImageNet mean and std
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # Crop the center 224x224 pixels
    transforms.ToTensor(),              # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

train_model_3_dataset = datasets.Flowers102(root='data', split='train', download=True, transform = transform_train)
val_model_3_dataset = datasets.Flowers102(root='data', split='val', download=True, transform = transform_test)
test_model_3_dataset = datasets.Flowers102(root='data', split='test', download=True, transform = transform_test)

train_model_3_dataloader = DataLoader(train_model_3_dataset, batch_size=32, shuffle=True)
val_model_3_dataloader = DataLoader(val_model_3_dataset, batch_size=32, shuffle=False)
test_model_3_dataloader = DataLoader(test_model_3_dataset, batch_size=32, shuffle=False)


In [ ]:
model_vgg16 = models.vgg16(pretrained=True)
num_ftrs = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(num_ftrs, 102)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg16.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Замораживаем все параметры модели
for param in model_vgg16.parameters():
    param.requires_grad = False

# Размораживаем только последние параметры для обучения
for param in model_vgg16.classifier.parameters():
    param.requires_grad = True

# model_vgg16.add(Dense(2,activation = 'softmax', name='output'))

optimizer = optim.SGD(model_vgg16.classifier.parameters(), lr=0.001,momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Setup TensorBoard logging
writer = SummaryWriter()


In [ ]:

def train_one_epoch(model, train_dataloader, criterion, optimizer, device, writer, epoch):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    all_train_preds = []
    all_train_labels = []

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
                        # Reset gradients
        outputs = model(inputs)  # Forward pass

        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimizer step

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        all_train_preds.extend(predicted.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct_preds / total_preds
    epoch_f1_score = f1_score(all_train_labels, all_train_preds, average='weighted')

    # Log to TensorBoard
    writer.add_scalar('Training Loss', epoch_loss, epoch)
    writer.add_scalar('Training Accuracy', epoch_accuracy, epoch)
    writer.add_scalar('Training F1 Score', epoch_f1_score, epoch)

    print(f'Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Train F1 Score: {epoch_f1_score:.4f}')

    scheduler.step()

    return epoch_loss, epoch_accuracy, epoch_f1_score

In [ ]:
def validate_one_epoch(model, val_dataloader, criterion, device, writer, epoch):
    model.eval()
    val_loss = 0.0
    correct_preds_val = 0
    total_preds_val = 0
    all_val_preds = []
    all_val_labels = []

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds_val += (predicted == labels).sum().item()
            total_preds_val += labels.size(0)

            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    val_loss = val_loss / len(val_dataloader.dataset)
    val_accuracy = correct_preds_val / total_preds_val
    val_f1_score = f1_score(all_val_labels, all_val_preds, average='weighted')

    writer.add_scalar('Validation Loss', val_loss, epoch)
    writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    writer.add_scalar('Validation F1 Score', val_f1_score, epoch)

    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1_score:.4f}')

    scheduler.step()

    return val_loss, val_accuracy, val_f1_score

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # Training step
    train_loss, train_accuracy, train_f1_score = train_one_epoch(
        model=model_vgg16,
        train_dataloader=train_model_3_dataloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        writer=writer,
        epoch=epoch
    )

    # Validation step
    val_loss, val_accuracy, val_f1_score = validate_one_epoch(
        model=model_vgg16,
        val_dataloader=val_model_3_dataloader,
        criterion=criterion,
        device=device,
        writer=writer,
        epoch=epoch
    )

# 11. **Close the TensorBoard writer**
writer.close()

Epoch 1, Train Loss: 4.5458, Train Accuracy: 0.0500, Train F1 Score: 0.0429
Epoch 1, Validation Loss: 4.0053, Validation Accuracy: 0.2598, Validation F1 Score: 0.2245
Epoch 2, Train Loss: 3.5692, Train Accuracy: 0.2794, Train F1 Score: 0.2492
Epoch 2, Validation Loss: 3.0184, Validation Accuracy: 0.5490, Validation F1 Score: 0.5062
Epoch 3, Train Loss: 2.4203, Train Accuracy: 0.5441, Train F1 Score: 0.5202
Epoch 3, Validation Loss: 2.0652, Validation Accuracy: 0.6147, Validation F1 Score: 0.5931
Epoch 4, Train Loss: 1.7241, Train Accuracy: 0.6696, Train F1 Score: 0.6569
Epoch 4, Validation Loss: 1.9387, Validation Accuracy: 0.6775, Validation F1 Score: 0.6592
Epoch 5, Train Loss: 1.5886, Train Accuracy: 0.7314, Train F1 Score: 0.7233
Epoch 5, Validation Loss: 1.8410, Validation Accuracy: 0.7098, Validation F1 Score: 0.6957
Epoch 6, Train Loss: 1.4722, Train Accuracy: 0.7716, Train F1 Score: 0.7655
Epoch 6, Validation Loss: 1.8331, Validation Accuracy: 0.7147, Validation F1 Score: 0.700

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs/flower_classification